In [1]:
# Standard library imports
import os
import sys
import copy
from typing import Union, Optional

# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import empyrical as ep

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

In [2]:
# Load data
path_to_data = "C:/Users/admin/OneDrive/Dokumente/Studium UZH/Master/5. Semester/Quantitative Portfolio Management with Python/Assingments/4. Assingment/1. Daten erhalten/" # <change this if necessary>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')
jkp_data = pd.read_parquet(path = f'{path_to_data}jkp_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
df1 = BacktestData()
df1.market_data = market_data # If the class "BacktestData" is used the attribute is always called "market_data" because it is defined like this within the class. If we want to use the function "get_return_series".
#print(df1.market_data.head(10))
#print(df1.market_data.tail(10))

df1.jkp_data = jkp_data
#print(df1.jkp_data.head(10))
#print(df1.jkp_data.tail(10))

# The dir() function returns a list of all attributes and methods of an object:
print(dir(df1)) # Return it for the entire class "BacktestData".
print(dir(df1.market_data)) # Return it for the specific DataFrame within the class "BacktestData".
print(dir(df1.jkp_data)) # Return it for the specific DataFrame within the class "BacktestData".
# These include:
# - Attributes (like data, market_data, etc.)
# - Methods (like get_return_series(), get_volume_series())
# - Special methods (like __init__, __str__, etc.)

# Get the return series:
returns = df1.get_return_series()
print(returns.head(10))
print(returns.tail(10))

# Get the volume series:
volumes = df1.get_volume_series()
print(volumes.head(10))
print(volumes.tail(10))

# Get the characteristic_series:
print(df1.jkp_data.columns.tolist())
z_scores = df1.get_characteristic_series(field="z_score")
print(z_scores.head(10))
print(z_scores.tail(10))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_characteristic_series', 'get_return_series', 'get_volume_series', 'jkp_data', 'market_data']
['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__arrow_c_stream__', '__bool__', '__class__', '__contains__', '__copy__', '__dataframe__', '__dataframe_consortium_standard__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__